In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Membaca data
data_path = '../dataset/supermarket_sales - Sheet1.csv'
sales_data = pd.read_csv(data_path)

# Cek dataset kosong
if sales_data.empty:
    print("Dataset kosong, tidak ada data untuk diproses.")
else:
    print(f"Dataset siap diproses dengan {sales_data.shape[0]} baris dan {sales_data.shape[1]} kolom.")

# Preprocessing: Tangani missing value
for column in sales_data.columns:
    if sales_data[column].dtype == 'object':  # Kolom non-numerik
        sales_data[column].fillna(sales_data[column].mode()[0], inplace=True)
    else:  # Kolom numerik
        sales_data[column].fillna(sales_data[column].median(), inplace=True)
        
# Cek jumlah baris duplikat
duplicate_rows = sales_data[sales_data.duplicated()]
print(f"Jumlah data duplikat: {duplicate_rows.shape[0]}")

# Cek kembali missing value
missing_values = sales_data.isnull().sum().sum()
if missing_values > 0:
    print(f"Ada {missing_values} missing values yang perlu ditangani.")
else:
    print("Tidak ada missing values dalam dataset.")

# Label encoding untuk 'Product line'
if 'Product line' in sales_data.columns:
    sales_data['Product_Label'] = sales_data['Product line'].astype('category').cat.codes

# Menampilkan grafik distribusi label
plt.figure(figsize=(8, 6))
sales_data['Product line'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribution of Product Line Labels')
plt.xlabel('Product Line')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Menambahkan kolom untuk analisis waktu
if 'Date' in sales_data.columns:
    sales_data['Date'] = pd.to_datetime(sales_data['Date'], errors='coerce')  # Tangani tanggal invalid
    sales_data['Month'] = sales_data['Date'].dt.month

# Menampilkan data keseluruhan (5 baris pertama)
print("Sample data:")
print(sales_data.head())

# Mengelompokkan data
monthly_sales = sales_data.groupby(['Product_Label', 'Month']).agg(
    Total_Quantity=('Quantity', 'sum'),
    Average_Unit_Price=('Unit price', 'mean')
).reset_index()

# Menampilkan data atribut yang digunakan dan jumlah kelas
print("Attributes used for training:", ['Month', 'Average_Unit_Price'])
print("Number of classes in Product_Label:", sales_data['Product_Label'].nunique())

# Menentukan fitur dan target
X = monthly_sales[['Month', 'Average_Unit_Price']]
y = monthly_sales['Total_Quantity']

# Standarisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Model KNN
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

# Prediksi
y_pred = knn.predict(X_test)

# Evaluasi menggunakan RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Akurasi
def accuracy(y_test, y_pred):
    return 1 - np.sqrt(np.mean((y_test - y_pred) ** 2)) / np.mean(y_test)

model_accuracy = accuracy(y_test, y_pred) * 100
print(f"Model Accuracy: {model_accuracy:.2f}%")

# Menambahkan bulan berikutnya untuk prediksi
next_month = monthly_sales['Month'].max() + 1
future_data = pd.DataFrame({
    'Month': [next_month] * monthly_sales['Product_Label'].nunique(),
    'Average_Unit_Price': monthly_sales.groupby('Product_Label')['Average_Unit_Price'].mean().values,
    'Product_Label': monthly_sales['Product_Label'].unique()
})

# Mapping label numerik kembali ke nama produk
product_names = sales_data[['Product_Label', 'Product line']].drop_duplicates()
future_data = future_data.merge(product_names, on='Product_Label', how='left')

# Standarisasi data bulan depan
future_data_scaled = scaler.transform(future_data[['Month', 'Average_Unit_Price']])

# Prediksi untuk bulan depan
future_data['Predicted_Quantity'] = knn.predict(future_data_scaled)

# Menambahkan kolom rekomendasi stok
future_data['Recommended_Quantity'] = np.ceil(future_data['Predicted_Quantity'] * 1.1)

# Simpan hasil prediksi dan rekomendasi ke file CSV
output_file_path = '../result/knn/predictions_and_recommendations_knn.csv'  
future_data.to_csv(output_file_path, index=False)
print(f"Hasil prediksi dan rekomendasi disimpan di {output_file_path}")

# Simpan RMSE dan akurasi ke file CSV
metrics = {
    'RMSE': [rmse],
    'Accuracy (%)': [model_accuracy]
}
metrics_df = pd.DataFrame(metrics)
metrics_file_path = '../result/knn/model_metrics_knn.csv' 
metrics_df.to_csv(metrics_file_path, index=False)
print(f"Metrik evaluasi disimpan di {metrics_file_path}")

# Visualisasi dengan dua grafik
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Grafik Predicted Quantity
rects1 = ax1.bar(future_data['Product line'], future_data['Predicted_Quantity'], color='blue')
ax1.set_title('Predicted Quantity by Product Line')
ax1.set_xlabel('Product Line')
ax1.set_ylabel('Predicted Quantity')
ax1.tick_params(axis='x', rotation=45)

# Menambahkan label pada setiap batang diagram (jumlah prediksi)
for rect in rects1:
    height = rect.get_height()
    ax1.annotate(f'{int(height)}',
                 xy=(rect.get_x() + rect.get_width() / 2, height),
                 xytext=(0, 3),  # Offset
                 textcoords="offset points",
                 ha='center', va='bottom')

# Grafik Recommended Quantity
rects2 = ax2.bar(future_data['Product line'], future_data['Recommended_Quantity'], color='orange')
ax2.set_title('Recommended Restocking Quantity by Product Line')
ax2.set_xlabel('Product Line')
ax2.set_ylabel('Recommended Quantity')
ax2.tick_params(axis='x', rotation=45)

# Menambahkan label pada setiap batang diagram (jumlah rekomendasi)
for rect in rects2:
    height = rect.get_height()
    ax2.annotate(f'{int(height)}',
                 xy=(rect.get_x() + rect.get_width() / 2, height),
                 xytext=(0, 3),  # Offset
                 textcoords="offset points",
                 ha='center', va='bottom')

plt.tight_layout()
plt.show()